Steps:
1. Prepare data by loading login info, ad content, and previously created clist dataframe
2. Use selenium to automate craigslist post
3. Loop through multiple email/password/location combinations to batch post ads

In [1]:
import pandas as pd
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#reload dataframe with cols: location, url, zip_code, ad_type
df=pd.read_pickle('clist_df')

In [3]:
#obtain login info and ad content
#*Note: user should define these variables directly with their own login info and ad content
emails=pickle.load(open('email_info','rb'))
passwords = pickle.load(open('pword_info','rb'))
post_title = pickle.load(open('post_title','rb'))
ad=pickle.load(open('ad_info','rb'))
phone_number = pickle.load(open('phone_info','rb'))
contact_name = pickle.load(open('contact_name','rb'))

In [4]:
#define college towns and highschool towns
col_towns = df[df['ad_type']=='col']
col=list(col_towns.index)
hs_towns = df[df['ad_type']=='hs']
hs= list(hs_towns.index)

In [5]:
#function to login and post to a particular location
#*note: user will need to change the first two xpaths to correspond with the specific ad type they would like to post
def post(loc, email, password):
    link=df['url'][loc]
    zipcode=df['zip_code'][loc]
    driver = webdriver.Chrome()
    driver.get(link)
    driver.find_element_by_id('post').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[10]/label/span[2]').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[13]/label/span[2]').click()
    driver.find_element_by_id('inputEmailHandle').send_keys(email)
    driver.find_element_by_id('inputPassword').send_keys(password)
    driver.find_element_by_xpath('/html/body/article/section/div/div[1]/form/div[3]/button').click()
    try:
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    except:
        #for those locations with an extra landing page, just choose the first sublocation
        #*idea: consider discovering ones w extra page and looping thru sublocations
        driver.find_element_by_xpath('/html/body/article/section/form/ul/li[1]/label').click()
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    driver.find_element_by_id('postal_code').send_keys(zipcode)
    driver.find_element_by_id('PostingBody').send_keys(ad)
    #*option: for longer ads, consider using copy and paste by uncommenting the line below
    # driver.find_element_by_id('PostingBody').send_keys(Keys.CONTROL, 'v')
    driver.find_element_by_xpath('//*[@id="contact_phone_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_text_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_phone"]').send_keys(phone_number)
    driver.find_element_by_xpath('//*[@id="contact_name"]').send_keys(contact_name)
    driver.find_element_by_xpath('//*[@id="postingForm"]/div/button').click()
    driver.find_element_by_xpath('//*[@id="leafletForm"]/button[1]').click()
    driver.find_element_by_xpath('/html/body/article/section/form/button').click()
    #*option: to automatically post without first reviewing the content, uncomment the line below
#     driver.find_element_by_xpath('//*[@id="publish_top"]/button').click()

In [6]:
#example function call
# i=2
# location='potsdam-massena'
# post(location, emails[i], passwords[i])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=60.0.3112.101)
  (Driver info: chromedriver=2.30.477700 (0057494ad8732195794a7b32078424f92a5fce41),platform=Windows NT 6.3.9600 x86_64)


Next steps:
3. Loop thru different emails and passwords for every two iterations (if i is multiple of three?) in order to post in the maximum of two locations per day per email. With only three email addresses, you can post to 6 locations per day so you will be able to cover all 30 in 5 days then it's just a matter of renewing. To get thru all 30 locations in one day, you would need 15 email addresses and 3 IP addresses (IP maxes out at 5 email accounts per IP).
1. Write another script to batch renew ads that were already posted